In [ ]:
import sys
import numpy as np
from tqdm import tqdm_notebook
import pickle
import pandas as pd
import plotly
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import seaborn as sns
import time
import init_common as x
import simulate_corona_multi as sc
import simulate_corona_senior_multi as scs
import multiprocessing as mp
import save_part as sp
import helpers as hp

## setting
alpha_option_type = 1# alpha: 0=all, 1=high, 2=med, 3=low, 
x.R0_type = 0
out_filename_root,out_fig_root,out_stat_root=hp.setup_paths(x.R0_type)
out_filename_pct0 = out_filename_root+"/"+x.out_filename_pct0
out_filename_raw0 = out_filename_root+"/"+x.out_filename_raw0

if x.R0_type==0:    
    import init_spec_gamma as xx
    import simulate_corona_multi as sf
elif x.R0_type==1:
    import init_spec_gamma as xx
    import simulate_corona_senior_multi as sf 

# print(out_filename_raw0)

## initialize
np.random.seed(123)
pct_data=[]
raw_data=[]

alpha_min = 0.5
alpha_max = 1
alpha_step = (alpha_max-alpha_min)/3

def run_simulation(alpha_vec,fun):
    ra = []
    rb = []
    cores_cnt = mp.cpu_count()
    pool = mp.Pool(cores_cnt)
    tic = time.perf_counter()
    res = pool.map(fun,alpha_vec)
    toc = time.perf_counter()
    pool.close()
    dt_avg = (toc-tic)/x.N_simul
    for k in np.arange(x.N_simul):
        ra0,rb0 = res[k]
        ra.append(ra0)
        rb.append(rb0)
    return ra,rb,dt_avg 

def save_results(filename):
    filename = filename+".pickle"
    fid = open(filename_pct,'wb')
    pickle.dump(pct_data,fid)
    fid.close()
    
opt_ok = True
if alpha_option_type == 0:
    alpha_val = np.arange(alpha_min,alpha_max+0.99*alpha_step,alpha_step)
    alpha_len = len(alpha_val)
    fnc_name = sf.simul
    file_ext = "_all"
elif alpha_option_type == 1: # high alpha
    alpha_val = x.public_trans_high
    fnc_name = sf.simul
    file_ext = "_high"
elif alpha_option_type == 2: # medium alpha
    alpha_val = x.public_trans_mid
    fnc_name = sf.simul
    file_ext = "_med"
elif alpha_option_type == 3: # low alpha
    alpha_val = x.public_trans_low
    fnc_name = sf.simul
    file_ext = "_low"
elif alpha_option_type == 4: # seniors
    alpha_val = x.public_trans_low
    fnc_name = sf.simul
    file_ext = "_low_senior"
else:
    print("Incorrect option")
    opt_ok = False

print("Option type: "+str(alpha_option_type))
print("R0:"+str(x.R0))
print("alpha:"+str(alpha_val))
print("beta:"+str(xx.beta))

In [ ]:
if opt_ok:
    if alpha_option_type>0:
        alpha_vec = np.full(x.N_simul,alpha_val)
    else:
        n = round(mp.cpu_count()/alpha_len)
        alpha_vec = np.repeat(alpha_val,n)
        

    ## run all simulations and 
    print("Running simulation ... ",end="",flush=True)
    pct_data,raw_data,tpi=run_simulation(alpha_vec,fnc_name)
    print("Finished")
    print("Time per iteration (sec.): "+str(tpi))

    ## save results
    print("Saving results ...",end="",flush=True)
    save_results(out_filename_pct0+file_ext)
    print("Finished")    
    

In [ ]:
## statistics
pct_stat_mean = []
pct_stat_std = []
n = round(mp.cpu_count()/alpha_len)
if alpha_option_type==0:
    for ia in range(alpha_len):
        y = pct_data[ia*n:(ia+1)*n]
        pct_stat_mean.append(hp.mean_list(y,'inf'))
        pct_stat_std.append(hp.std_list(y,'inf'))
    filename = out_stat_root+"/"+x.out_filename_pct0+file_ext+"_mean.pickle"
    fid = open(filename,'wb')
    pickle.dump(pct_stat_mean,fid)
    fid.close()
    filename = out_stat_root+"/"+x.out_filename_pct0+file_ext+"_std.pickle"
    fid = open(filename,'wb')
    pickle.dump(pct_stat_std,fid)
    fid.close()
    